<a href="https://colab.research.google.com/github/khavitidala/fuyukai-desu/blob/main/ulmfit/raw_lm_wiki_data_prep.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [1]:
!wget https://dumps.wikimedia.org/idwiki/latest/idwiki-latest-pages-articles.xml.bz2

--2021-11-25 02:50:14--  https://dumps.wikimedia.org/idwiki/latest/idwiki-latest-pages-articles.xml.bz2
Resolving dumps.wikimedia.org (dumps.wikimedia.org)... 208.80.154.7, 2620:0:861:1:208:80:154:7
Connecting to dumps.wikimedia.org (dumps.wikimedia.org)|208.80.154.7|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 725432365 (692M) [application/octet-stream]
Saving to: ‘idwiki-latest-pages-articles.xml.bz2’

idwiki-latest-pages 100%[===================>] 691.83M  4.93MB/s    in 2m 23s  

2021-11-25 02:52:39 (4.82 MB/s) - ‘idwiki-latest-pages-articles.xml.bz2’ saved [725432365/725432365]



In [7]:
!apt install bzip2 

Reading package lists... Done
Building dependency tree       
Reading state information... Done
bzip2 is already the newest version (1.0.6-8.1ubuntu0.2).
0 upgraded, 0 newly installed, 0 to remove and 37 not upgraded.


In [8]:
!bzip2 -d /content/drive/MyDrive/Riset/idwiki-latest-pages-articles.xml.bz2

In [16]:
!pip install wikiextractor

     |████████████████████████████████| 46 kB 1.7 MB/s 


In [19]:
!python -m wikiextractor.WikiExtractor /content/drive/MyDrive/Riset/idwiki-latest-pages-articles.xml --no-templates -o /content/drive/MyDrive/Riset/wiki/ --json

INFO: Starting page extraction from /content/drive/MyDrive/Riset/idwiki-latest-pages-articles.xml.
INFO: Using 1 extract processes.
INFO: Extracted 100000 articles (981.5 art/s)
INFO: Extracted 200000 articles (1368.1 art/s)
INFO: Extracted 300000 articles (1535.9 art/s)
INFO: Extracted 400000 articles (1442.9 art/s)
INFO: Extracted 500000 articles (1396.3 art/s)
INFO: Extracted 600000 articles (1485.4 art/s)
INFO: Extracted 700000 articles (1308.0 art/s)
INFO: Extracted 800000 articles (1962.7 art/s)
INFO: Extracted 900000 articles (1419.8 art/s)
INFO: Extracted 1000000 articles (1368.7 art/s)
INFO: Extracted 1100000 articles (1255.8 art/s)
INFO: Finished 1-process extraction of 1149642 articles in 837.9s (1372.1 art/s)


In [83]:
import json
import pathlib
import numpy as np
import pandas as pd
import html
import re
from fastai.text import *

In [2]:
from tqdm import tqdm

In [90]:
LANG = 'id'
PATH_ROOT = Path(f'/content/drive/MyDrive/Riset/')
PATH_EXTRACT = Path(f'{PATH_ROOT}/wiki/')
PATH_TMP = Path(f'{PATH_ROOT}/tmp/')

In [22]:
LANG_FILENAMES = [str(f) for f in PATH_EXTRACT.rglob("*/wiki_*")]
print(len(LANG_FILENAMES))
LANG_FILENAMES[0:5]

787


['/content/drive/MyDrive/Riset/wiki/AA/wiki_00',
 '/content/drive/MyDrive/Riset/wiki/AA/wiki_01',
 '/content/drive/MyDrive/Riset/wiki/AA/wiki_02',
 '/content/drive/MyDrive/Riset/wiki/AA/wiki_03',
 '/content/drive/MyDrive/Riset/wiki/AA/wiki_04']

In [36]:
LANG_TEXT = []
for fn in tqdm(LANG_FILENAMES):
    for line in open(fn, encoding='utf8'):
        LANG_TEXT.append(json.loads(line))
        
LANG_TEXT = pd.DataFrame(LANG_TEXT)

100%|██████████| 787/787 [00:12<00:00, 62.85it/s]


In [37]:
LANG_TEXT.head()

,id,revid,url,title,text
0,1,1223933,https://id.wikipedia.org/wiki?curid=1,Asam deoksiribonukleat,"Asam deoksiribonukleat, lebih dikenal dengan s..."
1,2,156791,https://id.wikipedia.org/wiki?curid=2,Asam Deoksiribosanukleat,
2,3,1218443,https://id.wikipedia.org/wiki?curid=3,Anwar Sadat,Muhammad Anwar el-Sadat; ) adalah seorang poli...
3,4,1119670,https://id.wikipedia.org/wiki?curid=4,Azhar Mansor,Datuk Azhar Mansor adalah orang Malaysia perta...
4,5,1217152,https://id.wikipedia.org/wiki?curid=5,Arkeologi,Arkeologi atau ilmu kepurbakalaan adalah ilmu ...


In [38]:
LANG_TEXT.shape

(1149641, 5)

In [39]:
# Getting rid of the title name in the text field
def split_title_from_text(text):
    words = text.split("\n\n")
    if len(words) >= 2:
        return ''.join(words[1:])
    else:
        return ''.join(words)
    
LANG_TEXT['text'] = LANG_TEXT['text'].apply(lambda x: split_title_from_text(x))

In [40]:
LANG_TEXT.head()

,id,revid,url,title,text
0,1,1223933,https://id.wikipedia.org/wiki?curid=1,Asam deoksiribonukleat,"Asam deoksiribonukleat, lebih dikenal dengan s..."
1,2,156791,https://id.wikipedia.org/wiki?curid=2,Asam Deoksiribosanukleat,
2,3,1218443,https://id.wikipedia.org/wiki?curid=3,Anwar Sadat,Muhammad Anwar el-Sadat; ) adalah seorang poli...
3,4,1119670,https://id.wikipedia.org/wiki?curid=4,Azhar Mansor,Datuk Azhar Mansor adalah orang Malaysia perta...
4,5,1217152,https://id.wikipedia.org/wiki?curid=5,Arkeologi,Arkeologi atau ilmu kepurbakalaan adalah ilmu ...


Sorting the articles by length and keeping the first million.

In [41]:
LANG_TEXT = LANG_TEXT.assign(length = 0)
LANG_TEXT.columns = ['id', 'revid','url', 'title', 'text', 'length']
LANG_TEXT = LANG_TEXT.assign(labels = 0).pipe(lambda x: x[['labels', 'text', 'length']])

In [42]:
LANG_TEXT.head()

,labels,text,length
0,0,"Asam deoksiribonukleat, lebih dikenal dengan s...",0
1,0,,0
2,0,Muhammad Anwar el-Sadat; ) adalah seorang poli...,0
3,0,Datuk Azhar Mansor adalah orang Malaysia perta...,0
4,0,Arkeologi atau ilmu kepurbakalaan adalah ilmu ...,0


In [45]:
LANG_TEXT['length'] = LANG_TEXT['text'].str.len()
LANG_TEXT = LANG_TEXT.sort_values(by=['length'], ascending=False)

In [46]:
LANG_TEXT.head()

,labels,text,length
1126518,0,Atala Riffo and Daughters v. Chile (bahasa Ind...,403071
12731,0,Sejarah Maritim Nusantara dan Tentara Nasional...,242121
5013,0,Josef Stalin (terlahir dengan nama Ioseb Besar...,149676
180009,0,Kemilau Cinta Kamila (disingkat KCK) adalah si...,141209
23021,0,"Di bidang historiografi, Romawi Kuno adalah se...",138582


In [61]:
LANG_TEXT.replace('', np.nan, inplace=True)

In [62]:
LANG_TEXT.isna().sum()

labels         0
text      555132
length         0
dtype: int64

In [63]:
LANG_TEXT.dropna(inplace=True)

In [64]:
len(LANG_TEXT)

594509

In [66]:
# LANG_TEXT.to_csv(f"{PATH_ROOT}/wiki_{LANG}_corpus.csv", header=False, index=False)

In [67]:
LANG_TEXT = LANG_TEXT[LANG_TEXT['length'] > 10]

In [68]:
len(LANG_TEXT)

594432

In [69]:
# Number of documents
print(LANG_TEXT['text'][:5])
LANG_TEXT.shape

1126518    Atala Riffo and Daughters v. Chile (bahasa Ind...
12731      Sejarah Maritim Nusantara dan Tentara Nasional...
5013       Josef Stalin (terlahir dengan nama Ioseb Besar...
180009     Kemilau Cinta Kamila (disingkat KCK) adalah si...
23021      Di bidang historiografi, Romawi Kuno adalah se...
Name: text, dtype: object


(594432, 3)

In [70]:
# Number of words in all the documents
def x_split(x):
    l = 0
    try:
        l = len(x.split(" "))
    except AttributeError:
        print(x)
    return l

LANG_TEXT['text'].apply(lambda x: x_split(x)).sum()

88763936

In [4]:
# LANG_TEXT = pd.read_csv(f"{PATH_ROOT}/wiki_{LANG}_corpus.csv")
# LANG_TEXT.columns = ['labels', 'text', 'length']
# LANG_TEXT = LANG_TEXT[LANG_TEXT['length'] > 10]

In [7]:
# Number of unique tokens across documents
len(set(''.join(LANG_TEXT['text'].values).split(" ")))

4584645

split each row into text files respectively

In [41]:
# clear plain text
re1 = re.compile(r'  +')

def fixup(x):
    x = x.replace('#39;', "'").replace('amp;', '&').replace('#146;', "'").replace(
        'nbsp;', ' ').replace('#36;', '$').replace('\\n', "\n").replace('quot;', "'").replace(
        '<br />', "\n").replace('\\"', '"').replace('<unk>','u_n').replace(' @.@ ','.').replace(
        ' @-@ ','-').replace('\\', ' \\ ')
    return re1.sub(' ', html.unescape(x))

In [49]:
def split(dft):
  with open('/content/drive/MyDrive/Riset/tmp/wiki/id/'+str(dft.name)+'.txt', 'w') as f:
    f.write(fixup(dft.text))

In [50]:
LANG_TEXT.apply(split, axis=1)

0         None
1         None
2         None
3         None
4         None
          ... 
594426    None
594427    None
594428    None
594429    None
594430    None
Length: 594431, dtype: object

In [62]:
pth = Path('/content/drive/MyDrive/Riset/tmp/wiki/id/')

In [69]:
len(pth.ls())

594431

In [ ]:
!tar cvf wiki_id_594431.tar /content/drive/MyDrive/Riset/tmp/wiki/id/

In [ ]:
!gzip /content/wiki_id_594431.tar

In [85]:
LANG_TEXT.drop(['labels', 'length'], axis=1, inplace=True)

In [87]:
LANG_TEXT_5 = LANG_TEXT[:500000]

In [91]:
LANG_TEXT_5.to_csv(f"{PATH_ROOT}/wiki_{LANG}_corpus_500000_preprocessed.csv", index=False)

In [93]:
ptt = Path(f"{PATH_ROOT}/")

In [94]:
ptt.ls()

[PosixPath('/content/drive/MyDrive/Riset/PKM-AI'),
 PosixPath('/content/drive/MyDrive/Riset/idwiki-latest-pages-articles.xml'),
 PosixPath('/content/drive/MyDrive/Riset/Computing'),
 PosixPath('/content/drive/MyDrive/Riset/riset_sunda'),
 PosixPath('/content/drive/MyDrive/Riset/idwiki-latest-pages-articles.csv'),
 PosixPath('/content/drive/MyDrive/Riset/wiki'),
 PosixPath('/content/drive/MyDrive/Riset/tmp'),
 PosixPath('/content/drive/MyDrive/Riset/wiki_id_corpus.csv'),
 PosixPath('/content/drive/MyDrive/Riset/wiki_id_corpus_500000_preprocessed.csv')]

In [95]:
!cp /content/drive/MyDrive/Riset/wiki_id_corpus_500000_preprocessed.csv /content/wiki_id_corpus_500000_preprocessed.csv

In [96]:
# lite version for easy debugging
LANG_TEXT_50 = LANG_TEXT[:50]
LANG_TEXT_50.to_csv(f"{PATH_ROOT}/wiki_{LANG}_corpus_50_preprocessed.csv", index=False)
LANG_TEXT_25 = LANG_TEXT_50[:25]
LANG_TEXT_25.to_csv(f"{PATH_ROOT}/wiki_{LANG}_corpus_25_preprocessed.csv", index=False)

In [97]:
!cp /content/drive/MyDrive/Riset/wiki_id_corpus_50_preprocessed.csv /content/wiki_id_corpus_50_preprocessed.csv

In [98]:
!cp /content/drive/MyDrive/Riset/wiki_id_corpus_25_preprocessed.csv /content/wiki_id_corpus_25_preprocessed.csv